# Workflows

## He3 Cell Workflow

The `he3` submodule provides a helper `He3CellWorkflow`:

In [ ]:
from ess import polarization as pol

print(pol.he3.He3CellWorkflow.__doc__)

### Opacity

There are two ways of computing the opacity, from cell parameters and from direct-beam measurements.

In [ ]:
workflow = pol.he3.He3CellWorkflow(in_situ=True)  # True is the default
workflow.visualize(pol.He3OpacityFunction[pol.Polarizer])

In [ ]:
workflow = pol.he3.He3CellWorkflow(in_situ=False)
workflow.visualize(pol.He3OpacityFunction[pol.Polarizer])

### Transmission function

The opacity can be used to obtain the transmission function of the cells.
We show the in-situ case, but it works equivalently for the more precise definition of opacity:

In [ ]:

workflow = pol.he3.He3CellWorkflow(in_situ=True, incoming_polarized=False)
workflow.visualize(
    pol.TransmissionFunction[pol.Polarizer], graph_attr={"rankdir": "LR"}
)

There is a variant of the workflow using an incoming polarized beam for computing the transmission function of the analyzer. 
Note that the incoming polarized case only applies to the analyzer, since for the polarizer the incoming neutron beam will always be unpolarized:

In [ ]:
workflow = pol.he3.He3CellWorkflow(in_situ=True, incoming_polarized=True)
workflow.visualize(
    (pol.TransmissionFunction[pol.Polarizer], pol.TransmissionFunction[pol.Analyzer]),
    graph_attr={"rankdir": "LR"},
)

## Supermirror Workflow

The `supermirror` submodule provides a helper `SupermirrorWorkflow`:

In [ ]:
from ess import polarization as pol

workflow = pol.supermirror.SupermirrorWorkflow()
print(workflow.__doc__)

In [ ]:
workflow.visualize(pol.TransmissionFunction[pol.Polarizer])

## Full pipeline

### On-the-fly reduction: opacity from cell parameters

In total, we can choose to use either He3-cells or supermirrors for both the polarizer and analyzer.
Following cell shows an example of using a polarizer supermirror and analyzer He3-cell with an incoming polarized beam and an in-situ opacity calculation:

In [ ]:
he3_workflow = pol.He3CellWorkflow(in_situ=True, incoming_polarized=True)
sm_workflow = pol.supermirror.SupermirrorWorkflow()

workflow = pol.PolarizationAnalysisWorkflow(
    analyzer_workflow=he3_workflow, polarizer_workflow=sm_workflow
)
results = (
    pol.PolarizationCorrectedData[pol.Up, pol.Up],
    pol.PolarizationCorrectedData[pol.Up, pol.Down],
    pol.PolarizationCorrectedData[pol.Down, pol.Up],
    pol.PolarizationCorrectedData[pol.Down, pol.Down],
)
workflow.visualize(results, graph_attr={"rankdir": "LR"})

### Precise reduction: opacity from direct-beam measurements of unpolarized cells

Here, an example is presented using the opacity calculation from direct-beam measurements (ex-situ case) for having 2x He3-cells as analyzer and polarizer. Note that the incoming-polarized beam only applies to the analyzer, since the polarizer will always use an incoming-unpolarized beam. Hence, one could alternatively use the same command is described under [Transmission function](#Transmission-function) for 2x He3-cells.


In [ ]:
he3_polarizer_workflow = pol.He3CellWorkflow(in_situ=False, incoming_polarized=False)
he3_analyzer_workflow = pol.He3CellWorkflow(in_situ=False, incoming_polarized=True)

workflow = pol.PolarizationAnalysisWorkflow(
    analyzer_workflow=he3_analyzer_workflow, polarizer_workflow=he3_polarizer_workflow
)
workflow.visualize(results, graph_attr={"rankdir": "LR"})

## Correction workflow

The correction worklow on its own looks as follows:

In [ ]:
from ess.polarization.correction import CorrectionWorkflow

workflow = CorrectionWorkflow()
workflow.visualize(results, graph_attr={"rankdir": "LR"})

There is also a half-polarized version of the workflow, in case only a polarizer is used:

In [ ]:
from ess.polarization.correction import CorrectionWorkflow

results = (
    pol.HalfPolarizedCorrectedData[pol.Up],
    pol.HalfPolarizedCorrectedData[pol.Down],
)
workflow = CorrectionWorkflow(half_polarized=True)
workflow.visualize(results, graph_attr={"rankdir": "LR"})

Note that in parts of the workflow `Up` is used as a dummy value for the spin of the analyzer that is not present.